In [ ]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal as ss

def voltage_criterion(data=None, v_thresh=-40, time_thresh=260):
    threshold_crossings = np.diff(data > v_thresh, prepend=False)
    upward_crossings = np.argwhere(threshold_crossings)[::2,0]
    downward_crossings = np.argwhere(threshold_crossings)[1::2,0]
    # If length of threshold_crossings is not even
    if np.mod(np.argwhere(threshold_crossings).reshape(-1,).shape[0],2)!=0:
        legit_up_crossings = upward_crossings[:-1][np.diff(np.argwhere(threshold_crossings).reshape(-1,))[::2]>time_thresh]
        legit_down_crossings = downward_crossings[np.diff(np.argwhere(threshold_crossings).reshape(-1,))[::2]>time_thresh]
    else:
        legit_up_crossings = upward_crossings[np.diff(np.argwhere(threshold_crossings).reshape(-1,))[::2]>time_thresh]
        legit_down_crossings = downward_crossings[np.diff(np.argwhere(threshold_crossings).reshape(-1,))[::2]>time_thresh]
    return upward_crossings, legit_up_crossings, legit_down_crossings

# Input: upward and downward crossings
# Output: bounds of NMDA spikes meeting current criterion
def current_criterion(legit_uc_iso=[], legit_dc_iso=[], p=1, control_inmda=np.array([1])):
    bounds = []
    sum_current = []
    

    for ind1 in np.arange(0,len(legit_uc_iso)):
        e1 = control_inmda[legit_uc_iso[ind1], p] #current @ up_crossing[ind1]
        #all the indices where current crosses 130% of e1
        x30 = np.argwhere(np.diff(control_inmda[legit_uc_iso[ind1]:legit_dc_iso[ind1], p] < 1.3*e1, prepend=False))
        #all the indices where current crosses 115% of e1
        x15 = np.argwhere(np.diff(control_inmda[legit_uc_iso[ind1]:legit_dc_iso[ind1], p] < 1.15*e1, prepend=False))
        
        if len(x30)>0:
        
            x15_copy = x15
            x30_copy = x30
            
            try:
                i = x30[0][0]
            except:
                import pdb; pdb.set_trace()
                
            n = 0
            
            
            while n==0:
                if len(np.sort(x15[x15>i]))!=0:
                    b1 = i
                    b2 = np.sort(x15[x15>i])[0]
                    bounds.append([legit_uc_iso[ind1]+b1,legit_uc_iso[ind1]+b2])
                    sum_current.append(np.sum(control_inmda[legit_uc_iso[ind1]+b1:legit_uc_iso[ind1]+b2,p])/10)
                else:
                    b1 = i
                    b2 = (legit_dc_iso[ind1]-legit_uc_iso[ind1])
                    bounds.append([legit_uc_iso[ind1]+b1,legit_uc_iso[ind1]+b2])
                    sum_current.append(np.sum(control_inmda[legit_uc_iso[ind1]+b1:legit_uc_iso[ind1]+b2,p])/10)
                    n=1
                
                x30_copy = x30_copy[x30_copy>legit_uc_iso[ind1]+b2]
                #import pdb; pdb.set_trace()
                
                if len(x30_copy)!=0:
                    i = x30_copy[x30_copy>b2][0]
                else:
                    n=1
    return bounds, sum_current

In [ ]:
%cd ../scripts/L5BaselineResults/

In [ ]:
conns = pd.read_csv('Connections.csv')

In [ ]:
segs = pd.read_csv('Segments.csv')
segs_degrees = pd.read_csv('SegmentsDegrees.csv').groupby(['Type','Sec ID'])['Degrees'].max().reset_index()
segs['segmentID'] = segs.index
segs = segs.set_index(['Type','Sec ID']).join(segs_degrees.set_index(['Type','Sec ID'])).reset_index()

segs['Sec ID'] = segs['Sec ID'].astype(int)
segs['X'] = segs['X'].astype(float)
segs['Elec_distanceQ'] = 'None'

segs.loc[segs.Type=='dend','Elec_distanceQ'] = pd.qcut(segs.loc[segs.Type=='dend','Elec_distance'], 10, labels=False)
segs.loc[segs.Type=='apic','Elec_distanceQ'] = pd.qcut(segs.loc[segs.Type=='apic','Elec_distance'], 10, labels=False)

In [ ]:
v = h5py.File('v_report.h5','r')
hva = h5py.File('Ca_HVA.ica_report.h5','r')
lva = h5py.File('Ca_LVAst.ica_report.h5','r')
ih = h5py.File('Ih.ihcn_report.h5','r')

nmda = h5py.File('inmda_report.h5','r')

na = h5py.File('NaTa_t.gNaTa_t_report.h5','r')
spks = h5py.File('spikes.h5','r')

In [ ]:
spktimes = spks['spikes']['biophysical']['timestamps'][:]
spkinds = np.sort((spktimes*10).astype(int))

get na spikes

In [ ]:
na_df = pd.DataFrame(columns=['segmentID','na_lower_bound'])

for p in segs[(segs.Type=='dend')|(segs.Type=='apic')].index:
    # print(p)
    threshold_crossings = np.diff(na['report']['biophysical']['data'][:,p] > 0.003, prepend=False)
    upward_crossings = np.argwhere(threshold_crossings)[::2,0]
    downward_crossings = np.argwhere(threshold_crossings)[1::2,0]
    # Only count if not within 2 ms after a somatic spike
    na_spks = [i for i in upward_crossings if ~np.any((i-spkinds>=-5) & (i-spkinds<50))]
    
#     if len(na_spks)>0:
#         na_df = na_df.append(pd.DataFrame({'segmentID':np.tile(p,len(na_spks)),
#                                                'na_lower_bound':na_spks}))
#     else:
#         na_df = na_df.append(pd.DataFrame({'segmentID':p,
#                                                'na_lower_bound':np.array([np.nan])}))
    
    #df.append is depracated; have to use pd.concat.
    if len(na_spks) > 0:
        new_rows = pd.DataFrame({'segmentID': np.tile(p, len(na_spks)),
                                 'na_lower_bound': na_spks})
    else:
        new_rows = pd.DataFrame({'segmentID': [p],
                                 'na_lower_bound': [np.nan]})
    
    na_df = pd.concat([na_df, new_rows], ignore_index=True)


In [ ]:
# get the value of the conductance at the peak
df = pd.DataFrame(columns=['segmentID','na_lower_bound','peak_value'])

for s in na_df[~pd.isnull(na_df.na_lower_bound)].segmentID.unique():

    inds = na_df[na_df.segmentID==s]['na_lower_bound'].astype(int).values
    
    L = np.split(na['report']['biophysical']['data'][:,s], inds)
    
    X = reshape_pad(L)
    
    peak_vals = np.max(X[:,0:10], axis=1)
    
    df = pd.concat((df,pd.DataFrame({'segmentID': s*np.ones((inds.shape[0],)).astype(int),
                                     'na_lower_bound': inds, 
                                     'peak_value': peak_vals[1:]})))

get ca spikes